In [ ]:
import math

In [ ]:

def swapV2(x, r1, r2, swapfee=0.003, useFloat=False):
	if useFloat:
		# As mathematical expression
		return r2 - r1*r2/(r1 + (1-swapfee)*x)
	else:
		# Copy Uniswap V2 code:
		# if type(x) != int or type(r1) != int or type(r2) != int:
		# 	print("Warning, non-integer input")
		amountInWithFee = x * 997

		numerator = amountInWithFee * r2
		
		denominator = r1 * 1000
		denominator += amountInWithFee
		amountOut = numerator // denominator
		return amountOut

In [ ]:

def tickFromSqrtPrice(sqrtPrice):
    # Calculated tick from SqrtPrice:
    calculatedTick = math.log(sqrtPrice)/math.log(math.sqrt(1.0001))
    calculatedTickFloor = math.floor(calculatedTick) 
    return calculatedTickFloor

def sqrtPriceFromTick(currentTick):
    sqrtPrice = math.ceil(math.sqrt(1.0001 ** currentTick))
    return sqrtPrice

def swapV3(sqrtPriceX96, liquidity, fee, spacing, inputAmount, zeroForOne):
    din = inputAmount - inputAmount*fee//10**6
    q96 = 2**96
    curTick = tickFromSqrtPrice(sqrtPriceX96)
    print("curTick", curTick)
    leftTick = (curTick//spacing)*spacing
    print("leftTick", leftTick)
    rightTick = leftTick + spacing
    print("rightTick", rightTick)

    leftPrice = sqrtPriceFromTick(leftTick)
    print("leftPrice", leftPrice)
    rightPrice = sqrtPriceFromTick(rightTick)
    print("rightPrice", rightPrice)


    if zeroForOne:
        num = liquidity*2**96
        product = din*sqrtPriceX96
        den = num + product
        price_next = math.ceil(num*sqrtPriceX96/den)
        print("price_next zfo", price_next)
        
        if price_next < leftPrice:
            price_next = leftPrice
    else:
        price_next = sqrtPriceX96 + math.ceil(q96*din/liquidity)
        print("price_next else", price_next)

        if price_next > rightPrice:
            price_next = rightPrice

    print("price_next", price_next)
    print("sqrtPriceX96", sqrtPriceX96)

    spax,spbx = (price_next,sqrtPriceX96) if zeroForOne else (sqrtPriceX96,price_next)
    dx = ((liquidity*2**96*(spbx-spax))//spbx)//spax
    print("dx", dx)
    dy = (liquidity*(spbx-spax))//2**96
    print("dy", dy)
    # return (dx,dy)
    if zeroForOne:
        return (dx,dy)
    else:
        return (dy,dx)

In [ ]:
# Pool V2:
# zeroForOne = True
# inAmount = 2722768
# outAmount = 747811405678518267894
# reserve0 = 21680170
# reserve1 = 6720212576627902105169
# res = swapV2(2722768, 21680170, 6720212576627902105169, swapfee=0.003, useFloat=False)
# print(res)

# Pool V3:
zeroForOne = True
inAmount = 3957993
outAmount = 2446484741464768
sqrtPriceX96 = 1970252748949935116187982154371584
liquidity = 1672155271144151122
fee = 500
spacing = 10
res = swapV3(sqrtPriceX96, liquidity, fee, spacing, inAmount, zeroForOne)
print(res)
